In [ ]:
from typing import *
from itertools import product

import numpy as np
import fastplotlib as fpl
import pygfx

In [ ]:
def make_circle(center: Tuple[float, float], radius: float, n_points: int = 50) -> np.ndarray:
    theta = np.linspace(0, 2 * np.pi, n_points)
    xs = radius * np.sin(theta)
    ys = radius * np.cos(theta)
    
    return np.column_stack([xs, ys]) + center

In [ ]:
spatial_dims = (1000, 1000)

circles = list()
for center in product(range(0, spatial_dims[0], 15), range(0, spatial_dims[1], 15)):
    circles.append(make_circle(center, 5, n_points=75))

In [ ]:
len(circles)

In [ ]:
temporal = list()

xs = np.arange(0, 10_000)
for i in range(len(circles)):
    if i % 2 == 0:
        ys = np.sin(xs) * 10
    else:
        ys = np.cos(xs) * 10
    
    temporal.append(ys)

In [ ]:
temporal = np.vstack(temporal)

In [ ]:
plot = fpl.GridPlot((1, 2))

contours = plot[0, 0].add_line(np.vstack(circles), thickness=3)
heatmap = plot[0, 1].add_heatmap(temporal)
selector = heatmap.add_linear_region_selector(axis="y")

plot.show()

In [ ]:
zero_alpha_ixs = list()

start_offset = 0
for c in circles:
    start_offset += c.shape[0]
    zero_alpha_ixs += [start_offset - 1, start_offset]
    
zero_alpha_ixs = zero_alpha_ixs[:-1]

In [ ]:
contours.colors[np.array(zero_alpha_ixs)] = [1, 1, 1, 0]

In [ ]:
def set_visible_alpha(ev):
    ixs_visible = ev.pick_info["selected_indices"]
    ixs_hide = np.setdiff1d(np.arange(len(circles)), ixs_visible)
    
    for i in ixs_visible:
        contours.world_object.geometry.colors.data[(i * 75) + 1:(i * 75) + 74, -1] = 1
        
    for i in ixs_hide:
        contours.world_object.geometry.colors.data[(i * 75) + 1:(i * 75) + 74, -1] = 0
        
    contours.world_object.geometry.colors.update_range()

In [ ]:
selector.bounds.add_event_handler(set_visible_alpha)